In [113]:
import wfdb, wfdb.processing, heartpy
from biosppy.signals import ecg
from numpy import ndarray
from pandas import read_csv as rcsv

import json, re
from time import time as t
from datetime import datetime as dt

from preproc_wfdb import show_ann_label, get_db, extract_ann, extract_rdheader

EDA on VFDB: https://www.kaggle.com/code/kooaslansefat/eda-on-mit-bih-malignant-ventricular-fibrill-db

In [2]:
# wfdb.io.show_ann_labels()
# wfdb.io.show_ann_classes()

In [3]:
# sample_dataset = rcsv("sample_p10_sensor_outputs.csv")
# (6000 / sample_dataset["HR (bpm)"]).tolist() # 6000 (in miliseconds)

In [4]:
def call_db_info(key : str, label : int, *args, **kwargs) -> dict:
    """
    Available DB key options:
        * key="vfdb" for MIT-BIH Malignant Ventricular Ectopy Database
        * key="nsrdb" for MIT-BIH Normal Sinus Rhythm Database
    """
    
    dataset = get_db(key=key, verbose=False)
    
    try:
        header = rcsv("data/{}-rdheader.csv".format(key))
    except:
        raise FileNotFoundError(
            "Please call wf_preproc.extract_rdheader() and " \
            "then name your file with data/{}-rdheader.csv".format(key)
        )
    
    return {
        "db" : dataset["db"],
        "records" : dataset["records"],
        "header" : header,
        "label" : label
    }

In [5]:
# annotations = rcsv("data/vfdb-rdann.csv")

# # preprocess the annotation label
# annotations.annot_aux = annotations.annot_aux.apply(
#     lambda x: x.replace('(', '').replace('NSR', 'N').replace('VFIB', 'VF')
# )

# # set annotations index "from" and "to"
# annotations.rename(columns={"annot_idx" : "annot_idx_from"}, inplace=True)
# annotations.insert(
#     column = "annot_idx_to",
#     loc = len(annotations.columns)-1,
#     value = annotations.groupby('r_name').annot_idx_from.shift(periods = -1, fill_value = -1).astype(int)
# )

# # remove last row from each r_name
# annotations = annotations[annotations.annot_idx_to != -1]

In [6]:
# r = wfdb.rdrecord(records[0], channels=[0, 1], pn_dir=db)
# a = wfdb.rdann(record_name=records[0], pn_dir=db, extension='atr')
# a.__dict__

In [7]:
# wfdb.plot_wfdb(record=r, annotation=a, time_units="samples")

* remember: f = N/t, where f = sampling frequency, N = signal length, t = time
* time_per_second = signal_len / sampling_freq = 525000/250 = 2100

In [8]:
# record_num = 418
# for _, i in annotations[annotations.r_name == record_num].iterrows():
#     if _ > 50:
#         FS = header[header.r_name == record_num].sampling_freq.squeeze()
#         from_idx = i.annot_idx_from
#         to_idx = i.annot_idx_to

#         ann = wfdb.rdann(record_name=str(record_num), sampfrom=from_idx, sampto=to_idx+FS, extension="atr", pn_dir=db)
#         print(ann.sample)
#         rr_interval = wfdb.processing.calc_rr(ann.sample, fs=FS)
#         rr_interval = np.insert(rr_interval, 0, ann.sample[0]) if from_idx == 1 else rr_interval
#             # np.insert() is used to add the first annotation sample (i.e., ann.sample[0]), only when annot_idx = 1
#         rr_interval = rr_interval # / ann.fs # normalize
#         print("f={}, t={} {}".format(from_idx, to_idx, rr_interval))

In [9]:
def get_record_start_partitions(
    record : str,
    db : str,
    min_duration : int,
    freq_rate : int,
    resampling : int = 0,
    max_total_partition : int = 10,
    *args, **kwargs
) -> dict:
    
    def get_label(annotation):
        p = re.compile("([A-Za-z]+)")
        return p.search(annotation)[1]
    
    start_sample = (min_duration * 60) * freq_rate # freq = N / time(s), therefore N = freq x time
    annotation = wfdb.rdann(record, 'atr', pn_dir=db, sampfrom=start_sample)
    signals, _ = wfdb.rdsamp(record, pn_dir=db, sampfrom=start_sample)
    
    if resampling != 0:
        start_sample = (min_duration * 60) * resampling
        signals, annotation = wfdb.processing.resample_multichan(
            signals, annotation, freq_rate, resampling)
    
    results = []
    if db.lower() == "vfdb":
        annotations = [get_label(a) for a in annotation.aux_note]
        annotations = list(map(lambda x: x.replace('NSR', 'N').replace('VFIB', 'VF'), annotations))
        positive_labels = ["VT", "VF", "VFL"]
        
        for ann, annot_sample in zip(annotations, annotation.sample):
            if ann in positive_labels:
                results.append(annot_sample-start_sample) # n sample before events
    
    elif db.lower() == "nsrdb":
        nsrdb_start_partition = 0
        
        for i in range(max_total_partition):
            results.append(nsrdb_start_partition)
            nsrdb_start_partition += start_sample
            
    else:
        raise ValueError("Available DB: 'vfdb' and 'nsrdb' only.")
    
    results = [r for r in results if r >= 0] # remove non-negative values
    return results

### Preprocessing (R-R Interval)

In [10]:
class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [11]:
class SignalPreprocessing():
    def __init__(self, record_idx : str, db : str, *args, **kwargs) -> None:
        super(SignalPreprocessing, self).__init__()
        self.record_idx = record_idx
        self.db = db
        
        # record
        self.r = wfdb.rdrecord(record_name=self.record_idx, pn_dir=self.db)
        
        # annotation
        self.a = wfdb.rdann(record_name=self.record_idx, pn_dir=self.db, extension='atr')
        
        self.signals = self.r.p_signal # can be multi-dimensional channels
        self.channels = self.r.sig_name
        self.freq_rate = self.r.fs
        
    def extract_rpeaks(self, signal, *args, **kwargs) -> ndarray:
        # segment
        rpeaks, = ecg.engzee_segmenter(
            signal=signal,
            sampling_rate=self.freq_rate
        )

        # correct R-peak locations
        rpeaks, = ecg.correct_rpeaks(
            signal=signal, rpeaks=rpeaks,
            sampling_rate=self.freq_rate, tol=.05
        )

        # extract templates
        _, rpeaks = ecg.extract_heartbeats(
            signal=signal, rpeaks=rpeaks,
            sampling_rate=self.freq_rate, before=.2, after=.4
        )
        
        return rpeaks
        
    def filter_signal(self, signal, low_freq, high_freq, *args, **kwargs) -> ndarray:
        return heartpy.filter_signal(
            signal, filtertype='bandpass',
            cutoff=[low_freq, high_freq], sample_rate=self.freq_rate
        )
    
    def get_peaklist(self,
        start_partitions : list,
        duration : int,
        label : int,
        resampling : int = 0,
        *args, **kwargs
    ) -> dict:
        '''
        Example return:
        return_example = {
            "peaks" : {
                1000 : [
                    {"channel" : "ECG1", "value" : [20, 30, 40]},
                    {"channel" : "ECG2", "value" : [50, 60, 70]},
                ],
                2500 : [
                    {"channel" : "ECG1", "value" : [55, 66, 77]},
                    {"channel" : "ECG2", "value" : [75, 85, 96]},
                ]
            }, 
            "created_at" : "2024-02-15", 
            "exc_time" : 50.184
        }
        '''
        
        # to calculate exc. time (in seconds)
        start_time = t()
        
        # reassign variables since they can be overrided if resampling != 0
        fr = self.freq_rate
        signals = self.signals
        
        # resampling signal (if any)
        if resampling != 0:
            resampled_signals, _ = wfdb.processing.resample_multichan(
                self.signals, self.a, fr, resampling
            )
            fr = resampling
            signals = resampled_signals
            
            # if you did resampling, partitions index should be adjusted
            # based on the newest max. signal length.
            start_partitions = [i for i in start_partitions if i <= len(signals)]
        
        FINAL_RESULTS, PARTITION = {}, {}
        for start_partition in start_partitions:
            delta = (duration * 60) * fr # remember, freq = N / time(s), therefore N = freq x time
            curr_signals = signals[start_partition:start_partition+delta, :]
            
            PEAKS_CHANNEL = []
            for i, channel in enumerate(self.channels):
                signal = curr_signals[:, i]
                signal = self.filter_signal(signal, 7, 30) # filtered
                peaks = self.extract_rpeaks(signal)
                peaks = [int(p) for p in peaks] # convert from np.int32 to INT
                PEAKS_CHANNEL.append(dotdict({"channel" : channel, "value" : peaks}))
            PARTITION[int(start_partition)] = PEAKS_CHANNEL
        
        FINAL_RESULTS["peaks"] = PARTITION
        FINAL_RESULTS["label"] = label
        FINAL_RESULTS["exc_time"] = round(t()-start_time, 3)
        FINAL_RESULTS["created_at"] = dt.now().strftime("%Y-%m-%d %X")
        
        return dotdict(FINAL_RESULTS)

In [12]:
import os

In [13]:
def create_dataset(verbose : bool = True, *args, **kwargs) -> None:
    db = kwargs["db"]
    records = kwargs["records"]
    label = kwargs["label"]
    
    for r in records:
        st = get_record_start_partitions(record=r, db=db, min_duration=5, freq_rate=128)
        proc = SignalPreprocessing(record_idx=r, db=db)
        peaklist_result = proc.get_peaklist(start_partitions=st, duration=5, label=label)
        
        PATH = "data/{}".format(db)
        if not os.path.exists(PATH):
            os.mkdir(PATH)
        
        with open("data/{}/{}.json".format(db, r), "w") as outfile: 
            json.dump(peaklist_result, outfile)
        
        if verbose:
            print("Record {} (label={}) was completed.".format(r, label))

In [14]:
create_dataset(**call_db_info("nsrdb", 0))

Record 16265 (label=0) was completed.
Record 16272 (label=0) was completed.
Record 16273 (label=0) was completed.
Record 16420 (label=0) was completed.
Record 16483 (label=0) was completed.
Record 16539 (label=0) was completed.
Record 16773 (label=0) was completed.
Record 16786 (label=0) was completed.
Record 16795 (label=0) was completed.
Record 17052 (label=0) was completed.
Record 17453 (label=0) was completed.
Record 18177 (label=0) was completed.
Record 18184 (label=0) was completed.
Record 19088 (label=0) was completed.
Record 19090 (label=0) was completed.
Record 19093 (label=0) was completed.
Record 19140 (label=0) was completed.
Record 19830 (label=0) was completed.


### Read files

In [15]:
from heartpy import analysis
import hrvanalysis as hrva
import os

In [19]:
def rr_preproc(rr_interval : list) -> list:
    nn_interval = hrva.remove_outliers(rr_intervals=rr_interval, verbose=False)

    # @param method: "malik", "kamath", "karlsson", "acar"
    nn_interval = hrva.remove_ectopic_beats(rr_intervals=nn_interval, method="malik", verbose=False)

    # @param interpolation_method: 'linear', 'time', 'index', 'values', 'nearest', 'zero', 'slinear',
    # 'quadratic', 'cubic', 'barycentric', 'krogh', 'spline', 'polynomial', 'from_derivatives',
    # 'piecewise_polynomial', 'pchip', 'akima', 'cubicspline'
    nn_interval = hrva.interpolate_nan_values(rr_intervals=nn_interval, interpolation_method="cubic")

    # remove NaN values which weren't filtered during interpolation; e.g., in the last index
    nn_interval = [i for i in nn_interval if str(i) != "nan"]
    
    return nn_interval

In [31]:
DF_RECORDS = []
dataset_pos_filepath = ["data/_vfdb/{}".format(p) for p in os.listdir("data/_vfdb") if p.endswith(".json")]
dataset_neg_filepath = ["data/_nsrdb/{}".format(p) for p in os.listdir("data/_nsrdb") if p.endswith(".json")]

for filepath in (dataset_pos_filepath+dataset_neg_filepath): # LOOP per file
    with open(filepath) as file:
        data = json.load(fp=file)

    for idx in data["peaks"].keys(): # LOOP per start_partition
        for ch_num, ch in enumerate(data["peaks"][idx]): # LOOP per channel
            peaklist = ch["value"]
            
            try:
                rr = analysis.calc_rr(peaklist=peaklist, sample_rate=128)
                nn_interval = rr_preproc(rr_interval=rr["RR_list"])

                FEATURES = {
                    "record_id" : int(filepath.split("/")[-1].split(".json")[0]),
                    "start_partition_idx" : idx,
                    "channel" : ch["channel"] + "_{}".format(str(ch_num))
                }

                # Reference:
                # 1. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5624990/
                # 2. https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html

                # TIME DOMAIN
                ftr_time_domain = hrva.get_time_domain_features(nn_interval)
                FEATURES.update(ftr_time_domain)

                ftr_geometric_time_domain = hrva.get_geometrical_features(nn_interval)
                FEATURES.update(ftr_geometric_time_domain)

                # Frequency Domain
                ftr_freq_domain = hrva.get_frequency_domain_features(nn_interval)
                FEATURES.update(ftr_freq_domain)

                # Non-linear Domain
                ftr_entropy = hrva.get_sampen(nn_interval) # sample entropy
                FEATURES.update({"entropy" : ftr_entropy["sampen"]})

                ftr_poincare = hrva.get_poincare_plot_features(nn_interval)
                FEATURES.update(ftr_poincare)

                # CVI (Cardiac Sympathetic Index), CSI (Cardiac Vagal Index)
                ftr_csi_cvi = hrva.get_csi_cvi_features(nn_interval)
                FEATURES.update(ftr_csi_cvi)

                FEATURES.update({"label" : int(data["label"])}) # set label
                DF_RECORDS.append(FEATURES)
            
            except Exception as e:
                print("File: {}\nError: {}".format(filepath, e))

In [182]:
from pandas import DataFrame as df, read_csv as rcsv

In [188]:
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import MinMaxScaler as mms
from sklearn.metrics import accuracy_score, auc, roc_curve, confusion_matrix as cfmat

from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [26]:
# vfdb_dataset = df(DF_RECORDS)
# vfdb_dataset.to_csv("data/features.csv", index=False)

In [196]:
if __name__ == "__main__":
    THIS_TIMESTAMP = "20231022"
    THIS_MODEL = "mlp"
    vfdb_dataset = rcsv("feature_store/{}_features.csv".format(THIS_TIMESTAMP))
    
    # set random state (seed)
    RAND_SEED = 43
    
    # feature selection
    X = vfdb_dataset[['mean_nni', 'rmssd', 'mean_hr', 'triangular_index', \
                      'total_power', 'csi', 'cvi', 'entropy']]
    y = vfdb_dataset.label
    
    # split dataset
    X_train, X_test, y_train, y_test = tts(X, y, test_size=.3, random_state=RAND_SEED)
    
    # scale the data
    scaler = mms()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

In [262]:
def evaluate(y_pred : ndarray, y_test : ndarray, *args, **kwargs) -> dict:
    start_time = t()
    
    tn, fp, fn, tp = cfmat(y_pred, y_test).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = (2 * precision * recall) / (precision + recall)
    fpr, tpr, threshold = roc_curve(y_test, y_pred)
    
    return {
        "accuracy" : float(accuracy_score(y_pred, y_test)),
        "true_pos" : int(tp),
        "true_neg" : int(tn),
        "false_pos" : int(fp),
        "false_neg" : int(fn),
        "recall" : float(recall),
        "precision" : float(precision),
        "f1_score" : float(f1_score),
        "false_pos_rate" : fpr.tolist(),
        "true_pos_rate" : tpr.tolist(),
        "auc_roc_threshold" : threshold.tolist(),
        "auc_roc_score" : float(auc(fpr, tpr)),
        "exc_time" : round(t()-start_time, 3),
        "created_at" : dt.now().strftime("%Y-%m-%d %X")
    }

In [237]:
import cpuinfo

In [202]:
model_selections = {
    "lr" : LR(random_state=RAND_SEED),
    "rf" : RFC(random_state=RAND_SEED),
    "svm" : SVC(random_state=RAND_SEED),
    "mlp" : MLPClassifier(
        hidden_layer_sizes=(16, 4),
        batch_size=64,
        learning_rate_init=1e-2,
        early_stopping=True,
        random_state=RAND_SEED
    )
}

In [263]:
start_training_time = t()
model_id = str(int(t())) + "-{}".format(THIS_MODEL)
model = model_selections[THIS_MODEL]
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

model_training_log = {
    "model_id" : model_id,
    "model_name" : THIS_MODEL,
    "model_version" : "1.0",
    "cloud_storage_uri" : "gs://",
    "evaluation" : evaluate(y_pred, y_test),
    "device_type" : "cpu",
    "device_name" : cpuinfo.get_cpu_info()['brand_raw'],
    "device_count" : int(1),
    "exc_time_sec" : t()-start_training_time,
    "data_snapshot_dt" : THIS_TIMESTAMP,
    "prc_dt" : dt.now().strftime("%Y-%m-%d %X")
}

In [275]:
# pickle.load(open(filepath, 'rb')).predict(X_test_scaled)

### Test with PyTorch (for neural nets)

In [73]:
import torch
from torch.utils.data import Dataset, DataLoader

In [74]:
class NeuralClassifier(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralClassifier,self).__init__()
        self.input_layer = torch.nn.Linear(input_dim, 256)
        self.hidden_layer1 = torch.nn.Linear(256, 128)
        self.hidden_layer2 = torch.nn.Linear(128, 64)
        self.hidden_layer3 = torch.nn.Linear(64, 32)
        self.output_layer = torch.nn.Linear(32, output_dim)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self,x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        out = self.relu(self.hidden_layer2(out))
        out = self.relu(self.hidden_layer3(out))
        out = self.output_layer(out)
        return self.sigmoid(out)

In [75]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]
 
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return self.length

In [76]:
# call model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralClassifier(input_dim=len(X.columns), output_dim=1)
model = model.to(device)

# set hyperparams
lr = 1e-5
bs = 64
num_epochs = 1000
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [77]:
X_train = torch.FloatTensor(X_train_scaled).to(device)
X_test = torch.FloatTensor(X_test_scaled).to(device)
y_train = torch.FloatTensor(y_train.tolist()).to(device)
y_test = torch.FloatTensor(y_test.tolist()).to(device)

In [78]:
trainset = dataset(X_train_scaled, y_train.tolist())
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)

testset = dataset(X_test_scaled, y_test.tolist())
testloader = DataLoader(testset, batch_size=bs, shuffle=False)

In [79]:
for epoch in range(num_epochs):
    for j, (X_train, y_train) in enumerate(trainloader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()

        #forward feed
        output_train = model(X_train)
        output_train = output_train.squeeze(1)

        #calculate the loss
        loss_train = criterion(output_train, y_train)

        acc_train = (output_train.round() == y_train).float().mean().item()

        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        output_test = model(X_test)
        output_test = output_test.squeeze(1)

        loss_test = criterion(output_test, y_test)
        acc_test = (output_test.round() == y_test).float().mean().item()

        # train_losses[epoch] = loss_train.item()
        # test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")
            print("Train acc: {:.3f}, Test acc: {:.3f}".format(acc_train, acc_test))
            print()

Epoch 50/1000, Train Loss: 0.6773, Test Loss: 0.6696
Train acc: 0.641, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6752, Test Loss: 0.6695
Train acc: 0.781, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6758, Test Loss: 0.6695
Train acc: 0.844, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6750, Test Loss: 0.6694
Train acc: 0.766, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6755, Test Loss: 0.6693
Train acc: 0.750, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6712, Test Loss: 0.6692
Train acc: 0.766, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6765, Test Loss: 0.6692
Train acc: 0.750, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6741, Test Loss: 0.6691
Train acc: 0.797, Test acc: 0.805

Epoch 50/1000, Train Loss: 0.6601, Test Loss: 0.6690
Train acc: 1.000, Test acc: 0.805

Epoch 100/1000, Train Loss: 0.6383, Test Loss: 0.6125
Train acc: 0.812, Test acc: 0.842

Epoch 100/1000, Train Loss: 0.6228, Test Loss: 0.6123
Train acc: 0.875, Test acc: 0.842

Epoch 100/1000, Train Loss: 0.

### Poincare and PSD plots

In [19]:
# from hrvanalysis import plot_psd, plot_poincare

# plot_poincare(nn_interval, plot_sd_features=True)
# plot_psd(nn_interval)